In [3]:
import torchvision.models as models


vgg16 = models.vgg16(pretrained=True)

In [4]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms

In [5]:
data_dir = r'C:\Users\lpapazian\Documents\GitHub\Post-Hurricane-Imagery-on-CNN'


transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((150,150))])

train = datasets.ImageFolder(data_dir + "/train_another",transform = transform)

#Validation
validation = datasets.ImageFolder(data_dir + "/validation_another",transform = transform)

In [6]:
#Chargement en DataLoader avec batch 128 pour éviter les crashs en entrainement et shuffle
#pour mélanger les batchs à chaque étape de l'entrainement
train_dl = torch.utils.data.DataLoader(train, 20, shuffle = True, num_workers = 4, pin_memory = True)

val_dl = torch.utils.data.DataLoader(validation, 20, shuffle = True, num_workers = 4, pin_memory = True)

In [7]:
original_dataset_dir = r'C:\Users\lpapazian\Documents\GitHub\Post-Hurricane-Imagery-on-CNN'


In [8]:


train_damage_dir = original_dataset_dir + '/train_another/damage'
validation_damage_dir = original_dataset_dir + '/validation_another/damage'
test_damage_dir = original_dataset_dir + '/test_another/damage'

train_nodamage_dir = original_dataset_dir + '/train_another/no_damage'
validation_nodamage_dir = original_dataset_dir + '/validation_another/no_damage'
test_nodamage_dir = original_dataset_dir + '/test_another/no_damage'



In [9]:
print('total training damage images: ',len(os.listdir(train_damage_dir)))
print('total validation damage images: ',len(os.listdir(validation_damage_dir)))
print('total test damage images: ',len(os.listdir(test_damage_dir)))

print('total training no damage images: ',len(os.listdir(train_nodamage_dir)))
print('total validation no damage images: ',len(os.listdir(validation_nodamage_dir)))
print('total test no damage images: ',len(os.listdir(test_nodamage_dir)))

total training damage images:  5000
total validation damage images:  1000
total test damage images:  8000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


In [10]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))


In [20]:

class ConvNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(32, 64, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(64, 128, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 128, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(128*7*7, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.Sigmoid())

    def forward(self, xb):
        return self.network(xb)

model = ConvNet()

In [22]:
#!pip install torchinfo
from torchinfo import summary
summary(model)

Defaulting to user installation because normal site-packages is not writeable


Layer (type:depth-idx)                   Param #
ConvNet                                  --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       896
│    └─ReLU: 2-2                         --
│    └─MaxPool2d: 2-3                    --
│    └─Conv2d: 2-4                       18,496
│    └─ReLU: 2-5                         --
│    └─MaxPool2d: 2-6                    --
│    └─Conv2d: 2-7                       73,856
│    └─ReLU: 2-8                         --
│    └─MaxPool2d: 2-9                    --
│    └─Conv2d: 2-10                      147,584
│    └─ReLU: 2-11                        --
│    └─MaxPool2d: 2-12                   --
│    └─Flatten: 2-13                     --
│    └─Linear: 2-14                      3,211,776
│    └─ReLU: 2-15                        --
│    └─Linear: 2-16                      1,026
│    └─Sigmoid: 2-17                     --
Total params: 3,453,634
Trainable params: 3,453,634
Non-trainable params: 0

In [11]:

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = []
    i = 0
    for batch in val_loader:
        outputs.append(model.validation_step(batch))
        i+=1
        if i == 50:
            break
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        i = 0
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            i+=1
            if i == 100:
                break
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history, 

In [13]:
num_epochs = 50
opt_func = torch.optim.RMSprop
lr = 1e-4
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 0.6649, val_loss: 0.5969, val_acc: 0.7340
Epoch [1], train_loss: 0.5604, val_loss: 0.5712, val_acc: 0.7380
Epoch [2], train_loss: 0.5448, val_loss: 0.5366, val_acc: 0.7630
Epoch [3], train_loss: 0.5129, val_loss: 0.4878, val_acc: 0.8320
Epoch [4], train_loss: 0.5090, val_loss: 0.6228, val_acc: 0.6670
Epoch [5], train_loss: 0.4953, val_loss: 0.5913, val_acc: 0.7060
Epoch [6], train_loss: 0.4851, val_loss: 0.5625, val_acc: 0.7260
Epoch [7], train_loss: 0.4991, val_loss: 0.4471, val_acc: 0.8710
Epoch [8], train_loss: 0.4563, val_loss: 0.5523, val_acc: 0.7380
Epoch [9], train_loss: 0.4576, val_loss: 0.4265, val_acc: 0.8940
Epoch [10], train_loss: 0.4341, val_loss: 0.4372, val_acc: 0.8700
Epoch [11], train_loss: 0.4470, val_loss: 0.4363, val_acc: 0.8670
Epoch [12], train_loss: 0.4295, val_loss: 0.4121, val_acc: 0.9020
Epoch [13], train_loss: 0.4237, val_loss: 0.4686, val_acc: 0.8400
Epoch [14], train_loss: 0.4099, val_loss: 0.4543, val_acc: 0.8530
Epoch [15], train_lo

In [52]:
from torchvision.models import resnet50, ResNet50_Weights

In [53]:
import torchvision
model_rn = torchvision.models.resnet18(pretrained=True)

In [54]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[[[ 2.3352e-02, -8.7704e-02,  1.8272e-01],
          [-1.6680e-01,  5.9087e-02,  1.7972e-01],
          [ 1.5056e-01,  9.7319e-02,  1.3996e-01]],

         [[ 9.4173e-03, -7.6684e-02, -8.6560e-02],
          [-2.4932e-03,  1.4716e-01,  1.2719e-01],
          [-5.4713e-02,  8.1617e-02, -1.6617e-01]],

         [[ 4.6618e-02, -1.6770e-01, -2.5300e-03],
          [ 6.3021e-02, -9.0441e-02, -1.1258e-01],
          [ 3.9303e-02,  1.8998e-01, -5.1503e-02]]],


        [[[-1.0081e-01, -6.3105e-02, -1.1734e-01],
          [ 8.6641e-02, -2.2529e-02,  1.0448e-01],
          [-7.5700e-02, -9.2915e-02,  9.6837e-02]],

         [[ 2.6904e-02, -1.6605e-01, -1.6504e-01],
          [-6.9071e-02,  1.4809e-01, -1.8625e-01],
          [-3.1581e-02, -1.1942e-01,  1.0671e-02]],

         [[ 1.4600e-01,  1.7060e-01, -1.3203e-02],
          [ 1.7543e-01, -1.9173e-01,  7.9418e-02],
          [ 3.0819e-02, -1.1667e-01,  1.9087e-02]]],


        [[[ 4.9614e-02,  2.6330e-03,  5.3669

In [55]:
model.parameters()

<generator object Module.parameters at 0x0000018FCAF624A0>

In [23]:
class ConvNet2(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size =(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(32, 64, kernel_size =(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(64, 128, kernel_size =(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 128, kernel_size =(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(128*9*9, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.Sigmoid())

    def forward(self, xb):
        return self.network(xb)

model2 = ConvNet2()

In [24]:
from torchsummary import summary
summary(model2, (3,150,150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 150, 150]             896
              ReLU-2         [-1, 32, 150, 150]               0
         MaxPool2d-3           [-1, 32, 75, 75]               0
            Conv2d-4           [-1, 64, 75, 75]          18,496
              ReLU-5           [-1, 64, 75, 75]               0
         MaxPool2d-6           [-1, 64, 37, 37]               0
            Conv2d-7          [-1, 128, 37, 37]          73,856
              ReLU-8          [-1, 128, 37, 37]               0
         MaxPool2d-9          [-1, 128, 18, 18]               0
           Conv2d-10          [-1, 128, 18, 18]         147,584
             ReLU-11          [-1, 128, 18, 18]               0
        MaxPool2d-12            [-1, 128, 9, 9]               0
          Flatten-13                [-1, 10368]               0
           Linear-14                  [

In [ ]:
num_epochs = 50
opt_func = torch.optim.Adam
lr = 1e-4
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model2, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 0.5431, val_loss: 0.5100, val_acc: 0.8030
Epoch [1], train_loss: 0.5203, val_loss: 0.4769, val_acc: 0.8390
Epoch [2], train_loss: 0.4848, val_loss: 0.4588, val_acc: 0.8610
Epoch [3], train_loss: 0.4483, val_loss: 0.4391, val_acc: 0.8710
Epoch [4], train_loss: 0.4435, val_loss: 0.4188, val_acc: 0.8950
Epoch [5], train_loss: 0.4480, val_loss: 0.4387, val_acc: 0.8690
Epoch [6], train_loss: 0.4277, val_loss: 0.4670, val_acc: 0.8430
Epoch [7], train_loss: 0.4263, val_loss: 0.4046, val_acc: 0.9090
Epoch [8], train_loss: 0.4280, val_loss: 0.4387, val_acc: 0.8740
Epoch [9], train_loss: 0.4236, val_loss: 0.4385, val_acc: 0.8710
Epoch [10], train_loss: 0.4145, val_loss: 0.3917, val_acc: 0.9200
